<a href="https://colab.research.google.com/github/NathanH-VCU/Access-Twitter-API/blob/main/bert_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip install -U tensorflow-text==2.6.0

!pip install tf-models-official==2.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 6.9 MB/s 
     |████████████████████████████████| 570.3 MB 316 bytes/s 
     |████████████████████████████████| 5.6 MB 30.2 MB/s 
     |████████████████████████████████| 1.3 MB 48.2 MB/s 
     |████████████████████████████████| 462 kB 48.2 MB/s 
     |████████████████████████████████| 132 kB 64.9 MB/s 
     |████████████████████████████████| 14.8 MB 36.3 MB/s 
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30694 sha256=f12475d809cf80cd8fad7787a1ce81d4f176ce01670eeb5d34b440c1901d405b
  Stored in directory: /root/.cache/pip/wheels/98/91/04/971b4c587cf47ae952b108949b46926f426c02832d120a082a
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68719 sha256=1a87717e72525101b127229fd908dfe840abb35e0dde7fdc7fef47f695374e4c
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 6.8 MB/s 
     |████████████████████████████████| 1.3 MB 47.7 MB/s 
     |████████████████████████████████| 99 kB 9.4 MB/s 
     |████████████████████████████████| 352 kB 34.7 MB/s 
     |████████████████████████████████| 238 kB 39.1 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 1.1 MB 51.2 MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=909ce6751f27bd1a29b2d8b7c2d75931072ad3d197c73d68eeb66aeaf34730e4
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=24b97a59a77a450e7d8747c5b83dd5704d2506dbda49a1e1dfabeee0e20f6cdb
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a

In [ ]:
import pandas as pd;
from scipy.stats import zscore
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import os

In [ ]:
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer


tf.get_logger().setLevel('ERROR')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
#read in the dataset

#url = 'https://raw.githubusercontent.com/nourchene-bargaoui/Kaggle_Dataset/main/comments.csv'
#dataset = pd.read_csv(url)

comments = pd.read_csv('https://raw.githubusercontent.com/nourchene-bargaoui/Kaggle_Dataset/main/comments.csv', error_bad_lines=False)
videos = pd.read_csv('https://raw.githubusercontent.com/nourchene-bargaoui/Kaggle_Dataset/main/videos-stats.csv', error_bad_lines=False)

In [ ]:
videos.head()

,Unnamed: 0,Title,Video ID,Published At,Keyword,Likes,Comments,Views
0,0,Apple Pay Is Killing the Physical Wallet After...,wAZZ-UWGVHI,2022-08-23,tech,3407.0,672.0,135612.0
1,1,The most EXPENSIVE thing I own.,b3x28s61q3c,2022-08-24,tech,76779.0,4306.0,1758063.0
2,2,My New House Gaming Setup is SICK!,4mgePWWCAmA,2022-08-23,tech,63825.0,3338.0,1564007.0
3,3,Petrol Vs Liquid Nitrogen | Freezing Experimen...,kXiYSI7H2b0,2022-08-23,tech,71566.0,1426.0,922918.0
4,4,Best Back to School Tech 2022!,ErMwWXQxHp0,2022-08-08,tech,96513.0,5155.0,1855644.0


In [ ]:
videos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1881 entries, 0 to 1880
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    1881 non-null   int64  
 1   Title         1881 non-null   object 
 2   Video ID      1881 non-null   object 
 3   Published At  1881 non-null   object 
 4   Keyword       1881 non-null   object 
 5   Likes         1879 non-null   float64
 6   Comments      1879 non-null   float64
 7   Views         1879 non-null   float64
dtypes: float64(3), int64(1), object(4)
memory usage: 117.7+ KB


In [ ]:
comments.head()

,Unnamed: 0,Video ID,Comment,Likes,Sentiment
0,0,wAZZ-UWGVHI,Let's not forget that Apple Pay in 2014 requir...,95.0,1.0
1,1,wAZZ-UWGVHI,Here in NZ 50% of retailers don’t even have co...,19.0,0.0
2,2,wAZZ-UWGVHI,I will forever acknowledge this channel with t...,161.0,2.0
3,3,wAZZ-UWGVHI,Whenever I go to a place that doesn’t take App...,8.0,0.0
4,4,wAZZ-UWGVHI,"Apple Pay is so convenient, secure, and easy t...",34.0,2.0


In [ ]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18409 entries, 0 to 18408
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  18409 non-null  int64  
 1   Video ID    18409 non-null  object 
 2   Comment     18408 non-null  object 
 3   Likes       18409 non-null  float64
 4   Sentiment   18409 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 719.2+ KB


In [ ]:
!pip install ktrain
!git clone https://github.com/nourchene-bargaoui/Kaggle_Dataset.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.3 MB 54.3 MB/s 
     |████████████████████████████████| 981 kB 48.6 MB/s 
     |████████████████████████████████| 263 kB 32.5 MB/s 
     |████████████████████████████████| 3.8 MB 44.5 MB/s 
     |████████████████████████████████| 468 kB 52.0 MB/s 
     |████████████████████████████████| 7.0 MB 42.7 MB/s 
     |████████████████████████████████| 163 kB 57.3 MB/s 
     |████████████████████████████████| 880 kB 41.4 MB/s 
  Created wheel for ktrain: filename=ktrain-0.31.9-py3-none-any.whl size=25312871 sha256=59ba4902197e9e341ba86ce9cefb130116bbd478679b2f281d21e4699c533cb0
  Stored in directory: /root/.cache/pip/wheels/35/f1/88/c7d6e7d0025eb5635327e62f12ca5fbb24e1502dd257a7aa77
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33517 sha256=3f1560cf218706e21bf898f5a524994cf38e135be36e399ef7118e27a3b282bc
  Stored in dire

In [ ]:
# Load data and set labels
data= pd.read_csv('/content/Kaggle_Dataset/comments.csv')
data.sample(5)

,Unnamed: 0,Video ID,Comment,Likes,Sentiment
2642,2642,On1iHuNBoMA,"Awesome work #Tshepiso, love the vision you go...",0.0,2.0
756,756,f2TK2lpFFzs,Iukikuik,0.0,1.0
13105,13105,YJ25XyrvuQ4,"""There will be more classified information in ...",36.0,0.0
6109,6109,B_LmCruLjis,I just want to share that these tips helped me...,51.0,2.0
9766,9766,1QKshmg0AlY,I love seeing mukbangers cook their own dishes...,69.0,2.0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = data.Comment.values
y = data.Sentiment.values
X_train, X_val, y_train, y_val =\
    train_test_split(X, y, test_size=0.1, random_state=2020)